## Generate MSTICpy Config

In [100]:
# Only generate config if Teams
# %%writefile msticpyconfig.yaml
# AzureSentinel:
#   Workspaces:
#     ASIHuntOMSWorkspaceV4:
#       TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
#       WorkspaceId: 52b1ab41-869e-4138-9e40-2a4457f09bf0
#     CyberSecuritySoc:
#       TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
#       WorkspaceId: 8ecf8077-cf51-4820-aadd-14040956f35d
#     Default:
#       TenantId: 72f988bf-86f1-41af-91ab-2d7cd011db47
#       WorkspaceId: 8ecf8077-cf51-4820-aadd-14040956f35d

## Core MSTICPy initialization for Notebooks

In [101]:
from msticpy.nbtools import nbinit
nbinit.init_notebook(namespace=globals())
qry_prov = QueryProvider("AzureSentinel")

In [102]:
qry_prov.connect(WorkspaceConfig())
tables = qry_prov.schema_tables

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Choose A Table To Analzye

In [103]:
import ipywidgets as widgets
from IPython.display import display

print('\nPlease select a table to analyze.\n')
tableDropdown = widgets.Dropdown(
    options=sorted(tables),
    value='OfficeActivity',
    description='Table:',
)
display(tableDropdown)


Please select a table to analyze.



Dropdown(description='Table:', index=231, options=('AACAudit', 'AACHttpRequest', 'AADDomainServicesAccountLogo…

## Choose a Feature(s) and Timeframe to Analyze

In [104]:
df = qry_prov.exec_query(f'{tableDropdown.value} | take 1')
timestamp_col = 'TimeGenerated'

if df.empty:
    print("Table is empty, please select another table.")
else:
    print ("\nWhat kind of features/columns would you like to see be analyzed?\n")
    options = sorted(list(df))
    options.remove(timestamp_col)
    try:
        options.remove('Type')
    except TypeError:
        print('Type does not exist')
    selected_features = nbwidgets.SelectSubset(source_items=options)
    print('\n\nWhat time frame do you want to analyze?\n')
    timeFrame = nbwidgets.QueryTime(units='day', max_before=20, before=5, max_after=1)
    timeFrame.display()

<IPython.core.display.Javascript object>


What kind of features/columns would you like to see be analyzed?





What time frame do you want to analyze?



## Generate Timeseries Anomalies

In [105]:
from datetime import timedelta
rawTimeRange = timeFrame.start - timedelta(hours=1)

In [265]:
from time_series_utils import check_kwargs, ts_anomalies_stl
from datetime import timedelta

start = (timeFrame.start)
end = (timeFrame.end)
# Timerange at which the raw data is retrieved. Default 1 hour before and 1 hour after timestamp.
rawTimeFrameStart = start - timedelta(hours=1)
rawTimeFrameEnd = end + timedelta(hours=1)
features = selected_features.selected_values

featureDict = {}
anomalyDict = {}
timeframeDict = {}

if start == end:
    print("\nPlease make sure the start and end date are distinct\n")
elif len(features) == 0:
    print("\nPlease choose at least one feature\n")
else:
    for feature in features:
        raw_times_series_data = qry_prov.MultiDataSource.get_timeseries_data(
            start=start,
            end=end,
            table=f"{tableDropdown.value}",
            timestampcolumn=timestamp_col,
            aggregatecolumn=f"{feature}",
            aggregatefunction=f"dcount(tostring({feature}))",
            add_query_items=f'|mv-expand {timestamp_col} to typeof(datetime), {feature} to typeof(long)',
        )
        df_time_series = raw_times_series_data[[timestamp_col, f'{feature}']]
        df_time_series = df_time_series.set_index(timestamp_col)
        anomalies = ts_anomalies_stl(df_time_series)
        featureDict[f'{feature}'] = anomalies
        anomalous_timestamps = list(anomalies[anomalies['anomalies']==1][timestamp_col])
        for timestamp in anomalous_timestamps:
            if timestamp not in anomalyDict:
                anomalyDict[timestamp] = [f'{feature}']
            else:
                anomalyDict[timestamp].append(f'{feature}')
        print(f'Timeseries for {feature} generated')
    print('\nQuerying Raw Data For ')
    print('\nTimestamp for Anomalous Features\n')
    anomalyDf = pd.DataFrame(anomalyDict.items(), columns=[timestamp_col, 'Anomalous Features'])
    anomalyDf = anomalyDf.sort_values(by=[timestamp_col]).reset_index(drop=True)
    anomalyTimeStamps = list(anomalyDf[timestamp_col])
    for timestamp in anomalyTimeStamps:
        # Timerange at which the raw data is retrieved. Default 1 hour before and 2 hours (including the 1 hour time range)
        startRange = timestamp - timedelta(hours=1)
        endRange = timestamp + timedelta(hours=2)
        anomalousFeatures = list(anomalyDf.loc[anomalyDf[timestamp_col] == timestamp]['Anomalous Features'])[0]
        queryString = f"{tableDropdown.value} \
        | where TimeGenerated >= datetime({startRange})\
        | where TimeGenerated <= datetime({endRange})\
        "
        result = qry_prov.exec_query(queryString)
        result = result.style.set_properties(**{'background-color': '#ffcccb'}, subset=anomalousFeatures)
        timeframeDict[timestamp] = result
    display(anomalyDf)

<IPython.core.display.Javascript object>

Timeseries for AADGroupId generated


<IPython.core.display.Javascript object>

Timeseries for AADTarget generated


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,TimeGenerated,Anomalous Features
0,2021-08-12 19:57:26.843325+00:00,"[SourceFileName, SourceFileName_]"
1,2021-08-13 13:57:26.843325+00:00,"[Site_, Site_Url, Site_Url_, SourceFileExtension, SourceFileName, SourceFileName_, SourceRelativ..."
2,2021-08-13 22:57:26.843325+00:00,[OriginatingServer]
3,2021-08-14 10:57:26.843325+00:00,"[ElevationTime, Start_Time]"
4,2021-08-15 07:57:26.843325+00:00,"[OfficeId, OfficeObjectId, Operation, Parameters, SourceRecordId]"
5,2021-08-15 21:57:26.843325+00:00,"[ElevationTime, Folders, Start_Time]"
6,2021-08-16 08:57:26.843325+00:00,"[OfficeId, OfficeObjectId, Operation, Parameters, SourceRecordId]"
7,2021-08-17 12:57:26.843325+00:00,"[OfficeId, OfficeObjectId, Operation, Parameters, SourceRecordId]"
8,2021-08-17 15:57:26.843325+00:00,"[AppId, ClientAppId, Client_IPAddress, ElevationTime, LogonUserSid, MailboxGuid, MailboxOwnerSid..."
9,2021-08-17 18:57:26.843325+00:00,"[ItemType, SourceFileName, SourceFileName_]"


## Select A Timestamp To Visualize

In [266]:
try:
    timestamp_dropdown = widgets.Dropdown(
        options=sorted(list(anomalyDf[timestamp_col])),
        description='TimeStamp:',
        disabled=False,
    )
    print('\nSelect a timestamp to visualize anomalous features ')
    display(timestamp_dropdown)
except NameError:
    print("Anomaly Dataframe not instantiated yet.")


Select a timestamp to visualize anomalous features 


Dropdown(description='TimeStamp:', options=(Timestamp('2021-08-12 19:57:26.843325+0000', tz='UTC'), Timestamp(…

## Visualize Anomalies

In [267]:
from msticpy.nbtools.timeseries import display_timeseries_anomolies
pd.set_option("display.max_rows", None, "display.max_columns", None)

try:
    anomalous_features = tuple(anomalyDf.loc[anomalyDf[timestamp_col] == timestamp_dropdown.value]['Anomalous Features'])[0]
    for feature in anomalous_features:
        print(f"\nTime Series for {feature}\n")
        display_timeseries_anomolies(data=featureDict[feature], y=feature)
    df = timeframeDict[timestamp_dropdown.value]
    display(timeframeDict[timestamp_dropdown.value])
except NameError:
    print("Anomaly Dataframe not instantiated yet.")


Time Series for SourceFileName



Loading BokehJS ...


Time Series for SourceFileName_



Loading BokehJS ...

,TenantId,Application,UserDomain,UserAgent,RecordType,TimeGenerated,Operation,OrganizationId,OrganizationId_,UserType,UserKey,OfficeWorkload,ResultStatus,ResultReasonType,OfficeObjectId,UserId,UserId_,ClientIP,ClientIP_,Scope,Site_,ItemType,EventSource,Source_Name,MachineDomainInfo,MachineId,Site_Url,Site_Url_,SourceRelativeUrl,SourceRelativeUrl_,SourceFileName,SourceFileName_,SourceFileExtension,DestinationRelativeUrl,DestinationFileName,DestinationFileExtension,UserSharedWith,SharingType,CustomEvent,Event_Data,ModifiedObjectResolvedName,Parameters,ExternalAccess,OriginatingServer,OrganizationName,Logon_Type,InternalLogonType,MailboxGuid,MailboxOwnerUPN,MailboxOwnerSid,MailboxOwnerMasterAccountSid,LogonUserSid,LogonUserDisplayName,ClientInfoString,Client_IPAddress,ClientMachineName,ClientProcessName,ClientVersion,Folder,CrossMailboxOperations,DestMailboxId,DestMailboxOwnerUPN,DestMailboxOwnerSid,DestMailboxOwnerMasterAccountSid,DestFolder,Folders,AffectedItems,Item,ModifiedProperties,SendAsUserSmtp,SendAsUserMailboxGuid,SendOnBehalfOfUserSmtp,SendonBehalfOfUserMailboxGuid,ExtendedProperties,Client,LoginStatus,Actor,ActorContextId,ActorIpAddress,InterSystemsId,IntraSystemId,SupportTicketId,TargetContextId,DataCenterSecurityEventType,EffectiveOrganization,ElevationTime,ElevationApprover,ElevationApprovedTime,ElevationRequestId,ElevationRole,ElevationDuration,GenericInfo,SourceSystem,OfficeId,SourceRecordId,AzureActiveDirectory_EventType,AADTarget,Start_Time,OfficeTenantId,OfficeTenantId_,TargetUserOrGroupName,TargetUserOrGroupType,MessageId,Members,TeamName,TeamGuid,ChannelType,ChannelName,ChannelGuid,ExtraProperties,AddOnType,AddonName,TabType,Name,OldValue,NewValue,ItemName,ChatThreadId,ChatName,CommunicationType,AADGroupId,AddOnGuid,AppDistributionMode,TargetUserId,OperationScope,AzureADAppId,OperationProperties,AppId,ClientAppId,Type,_ResourceId
0,8ecf8077-cf51-4820-aadd-14040956f35d,,,,50,2021-08-12 19:17:51+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,10032000876DFA4A,Exchange,Succeeded,Succeeded,,brandon@seccxpninja.onmicrosoft.com,brandon@seccxpninja.onmicrosoft.com,,,,,,,,,,,,,,,,,,,,,,,,,,False,BL3P221MB0386 (15.20.4200.000),seccxpninja.onmicrosoft.com,Owner,0.000000,41cab07f-6534-4c5d-bf85-62419b98d625,brandon@seccxpninja.onmicrosoft.com,S-1-5-21-1215315387-180893480-700816437-13930588,,S-1-5-21-1215315387-180893480-700816437-13930588,,Client=REST;Client=RESTSystem;;,2603:10b6:208:355::7,,,,,None,,,,,,"[{""FolderItems"":[{""InternetMessageId"":""<0d7c5f89-6871-4898-b3d5-791e61a27443@az.southcentralus.production.microsoft.com>""},{""InternetMessageId"":""""},{""InternetMessageId"":""""},{""InternetMessageId"":""""},{""InternetMessageId"":""<00740a42-a760-447a-80fd-bfb44b9c88e6@az.northcentralus.production.microsoft.com>""},{""InternetMessageId"":""<8c1a721f-36d1-4f83-bc2c-3e0655e04c8f@az.southcentralus.production.microsoft.com>""}],""Id"":""LgAAAABGXwQvA9OlRa9qOnOgscCkAQBd3HwPYoVGS59CpR/8NsKSAAAAAAEMAAAB"",""Path"":""\\Inbox""}]",,,,,,,,,,nan,,,,,,,,nan,,2021-08-12 19:30:29+00:00,,NaT,,,nan,,OfficeActivityManager,8df366dd-34a0-4825-a646-19f5e6765e12,8df366dd-34a0-4825-a646-19f5e6765e12,,,2021-08-12 19:30:29+00:00,$RestApiTenantId$,$RestApiTenantId$,,,,None,,,,,,None,,,,,,,,,,,,,,,,,"[{'Value': 'Bind', 'Name': 'MailAccessType'}, {'Value': 'False', 'Name': 'IsThrottled'}]",8a18aa92-0a1e-4e06-abd0-e118fa4787b1,3c8e478f-21ca-493a-b87c-c7366d664d54,OfficeActivity,
1,8ecf8077-cf51-4820-aadd-14040956f35d,,,,ExchangeAdmin,2021-08-12 19:24:06+00:00,Set-User,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,DcAdmin,NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),Exchange,True,True,NAMPR06A007.PROD.OUTLOOK.COM/Microsoft Exchange Hosted Organizations/seccxpninja.onmicrosoft.com/Yanivsh,NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),,,,,,,,,,,,,,,,,,,,